In [6]:
import os
import tensorflow as tf

In [2]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"

## NASBench-101

### Search Space Data Preprocessing

### Oracle: An Upper Bound Baseline

In [ ]:
!curl -O https://storage.googleapis.com/nasbench/nasbench_full.tfrecord

from nasbench import api
nasbench = api.NASBench('nasbench_full.tfrecord')

In [ ]:
# Download the raw data (only 108 epoch data points, for full dataset,
# uncomment the second line for nasbench_full.tfrecord).

if not os.path.exists('nasbench_full.tfrecord'):
  !curl -O https://storage.googleapis.com/nasbench/nasbench_full.tfrecord

# Initialize the NASBench object which parses the raw data into memory (this
# should only be run once as it takes up to a few minutes).
from nasbench import api

# Use nasbench_full.tfrecord for full dataset (run download command above).
nasbench = api.NASBench('nasbench_full.tfrecord')

In [ ]:
import random
import numpy as np
from tqdm import tqdm

full_metrics = nasbench.computed_statistics

In [ ]:
# Best Performance Test
best_val_acc = []
best_test_acc = []

for _, full_metric in tqdm(full_metrics.items()):
  for _, metric in full_metric.items():
    avg_val_acc = np.average([m['final_validation_accuracy'] for m in metric])
    avg_test_acc = np.average([m['final_test_accuracy'] for m in metric])

  best_val_acc.append(max_val_acc)
  best_test_acc.append(max_test_acc)

In [ ]:
max_val_acc = np.amax(best_val_acc)
corr_test_acc = best_test_acc[np.argmax(best_val_acc)]

max_test_acc = np.amax(best_test_acc)
corr_val_acc = best_val_acc[np.argmax(best_test_acc)]

print("Best Performance")
print(f"\033[1mval: {max_val_acc}\033[0m test: {corr_test_acc}")
print(f"val: {corr_val_acc} \033[1mtest: {max_test_acc}\033[0m")

In [ ]:
# Variance Test (will be used as oracle)
val_acc = []
test_acc = []

# Repeat oracle method for 100 times
for _ in tqdm(range(100)):
  max_val_acc = 0
  max_test_acc = 0

  for _, full_metric in full_metrics.items():
    for _, metric in full_metric.items():
      sample_metric = random.choice(metric)
      sample_val_acc = sample_metric['final_validation_accuracy']
      sample_test_acc = sample_metric['final_test_accuracy']

      if sample_val_acc > max_val_acc:
        max_val_acc = sample_val_acc
      if sample_test_acc > max_test_acc:
        max_test_acc = sample_test_acc

  val_acc.append(max_val_acc)
  test_acc.append(max_test_acc)

In [ ]:
avg_val_acc = np.average(val_acc)
std_val_acc = np.std(val_acc)

avg_test_acc = np.average(test_acc)
std_test_acc = np.std(test_acc)

print("\nBest Validation Accuracy")
print(f"avg: {avg_val_acc} std: {std_val_acc}")

print("\nBest Test Accuracy")
print(f"avg: {avg_test_acc} std: {std_test_acc}")

### Random Search: A Lower Bound Baseline

In [ ]:
from random_search import run_random_search

# Change the experiment setting here
# 600 runs, time limit set to 8e6 according to the paper
LOOPS = 600
MAX_TIME_BUDGET = 8e6

times = []
best_vals = []
best_tests = []

for _ in tqdm(range(LOOPS)):
    time, best_val, best_test = run_random_search(nasbench, max_time_budget=MAX_TIME_BUDGET)
    times.append(time)
    best_vals.append(best_val)
    best_tests.append(best_test)

times = np.average(np.array(times), axis=0)
best_vals = np.average(np.array(best_vals), axis=0)
best_tests = np.average(np.array(best_tests), axis=0)

### Neural Predictor (Two Stages)

### Neural Predictor (Single Stage)

### N vs K and Ablation Study